## Робота з мережею

In [1]:
print("лекція 2, робота з мережею")

лекція 2, робота з мережею


### Основи організації комп'ютерних мереж
---

Вимоги до великої комп'ютерної мережі:
- надійна, продовжує працювати при відключенні якомога більшої частки вузлів мережі;
- сумісна з різноманітним обладнанням та програмним забезпеченням;
- може легко розширюватися;
- забезпечує необхідну якість обслуговування і керування рівнем обслуговування;
- забезпечувати необхідний рівень безпеки;
- ...
- ...

Навіть неповний перелік вимог (а він неповний) — є досить складним. Основою вирішення складних завдань є __ДЕКОМПОЗИЦІЯ__. Саме так і вирішується питання реалізації переліку усіх складних вимог — всі вимоги розбиваються на функціональні групи та кожна вирішується різноманітними протоколами на різних рівнях. Докладніше — на малюку.

![ієрархія мережевих протоколів, принци](media/net_protocols_b.png)

__Інтерфейс__ — набір примітивних операцій, які нижній рівень надає верхньому.
__Протокол__ — набір узгоджених правил, які використовуються для зв'язку рівня N одного комп'ютеру з рівнем N іншого комп'ютеру.

На сьогодні є декілька моделей мережевої взаємодії. Однією з основних теоретичних моделей мережевої взаємодії є семирівнева [модель OSI (Open Systems Interconnection)](https://ru.wikipedia.org/wiki/%D0%A1%D0%B5%D1%82%D0%B5%D0%B2%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C_OSI). Вона теоретична, але добре опрацьована та детально спланована.


![мережева модель OSI](media/OSI.png)

### TCP/IP

![TCP/IP model](media/tcp_ip.png)


### Транспортний рівень

#### Завдання, які вирішує транспортний рівень

![Завдання транспортного рівня TCP/IP](media/tcp_udp.png)


Відповідно до завдань і складності протоколів, що обумовлено складнішою процедурою встановлення з'єднання для TCP та "більш складною мовою" між учасниками з'єднання. Порівняйте складність заголовків для пакетів даних для двох наведених протоколів:

![headers: tcp VS udp](media/headers_udp_VS_tcp.png)


Коли пишеться код, який повинен приймати\передавати якісь дані від іншого коду — зазвичай використовують технологію, що має назву сокети ([sockets, berekley sockets](https://en.wikipedia.org/wiki/Berkeley_sockets)). Це не єдина технологія для зв'язку між процесами, але досить поширена.

Сокети та API сокетів використовуються для надсилання повідомлень за допомогою мережі. Вони забезпечують форму міжпроцесного зв’язку (IPC). Мережа може бути логічною локальною мережею для комп’ютеру або фізично під’єднаною до зовнішньої мережі з власними з’єднаннями з іншими мережами. Яскравим прикладом є Інтернет, до якого ви підключаєтеся через свого провайдера.

Sockets мають довгу історію. Їх використання почалося з ARPANET у 1971 році, а пізніше стало API в операційній системі Berkeley Software Distribution (BSD), випущеною у 1983 році під назвою Berkeley sockets.

Сьогодні, незважаючи на те, що базові протоколи, які використовуються API сокетів, розвивалися протягом багатьох років та з’являлися нові, низькорівневий API залишився незмінним.

Найпоширенішим типом socket-додатків є клієнт-серверні додатки, де одна сторона діє як сервер та очікує підключення від клієнтів. Протягом заняття ми зосередимося на API для Інтернет-сокетів, які іноді називають сокетами Berkeley або BSD. Існують також доменні сокети Unix, які можна використовувати лише для зв’язку між процесами на одному хості.


### Sockets

У Python є вбудований модуль socket, який забезпечує API сокетів. Ми будемо його використовувати для створення простого ехо-сервера та клієнта, для демонстрації основних принципів роботи з сокетами. Докладна документація за модулем socket [тут](https://docs.python.org/3/library/socket.html).


#### TCP-сокети

Ми збираємося створити об’єкт сокету за допомогою socket.socket(), вказавши тип сокета як socket.SOCK_STREAM. Коли ми це робимо, протоколом за замовчуванням використовується Transmission Control Protocol (TCP). Це добрий параметр за замовчуванням.

Чому варто використовувати TCP в цьому випадку? Протокол керування передачею (TCP):

__Надійний:__ пакети, що потрапляють у мережу, виявляються та повторно передаються відправником.
__Контролює порядок доставки даних__: дані зчитуються вашою програмою в тому порядку, в якому вони були записані відправником.

Навпаки, сокети протоколу користувацьких дейтаграм (UDP), створені за допомогою socket.SOCK_DGRAM, ненадійні, і дані, зчитані одержувачем, можуть не відповідати порядку запису відправника.

Чому це важливо? Мережі — це найкраща система доставки. Немає гарантії, що ваші дані досягнуть місця призначення або що ви отримаєте те, що вам надіслали.

Мережеві пристрої, такі як маршрутизатори та комутатори, мають обмежену пропускну здатність та мають власні системні обмеження. Вони мають процесори, пам’ять, шини та буфери пакетів інтерфейсу, як ваші клієнти та сервери. TCP звільняє вас від необхідності турбуватися про втрату пакетів, неправильне надходження даних та інші підводні камені, які незмінно трапляються під час спілкування в мережі.

Але ви повинні розуміти, що для кожного конкретного випадку треба розглядати можливість використання максимально простих протоколів, які відповідають завданню. Наприклад — UDP протокол простий та для його реалізації необхідно значно менше процесорних і мережевих ресурсів. Тому при вирішенні завдань, які генерують значні обсяги даних й не є чутливими до втрати якоїсь частки пакетів — цілком природно використовувати UDP (наприклад — для передачі відео\аудіо потоків).

![socket flow](media/socket_flow.png)


Ліва колонка представляє сервер. З правого боку — клієнт.

Починаючи з верхнього лівого стовпця, зверніть увагу на виклики API, які сервер робить для налаштування «прослуховувального» сокету:

- socket() — створити сокет;
- bind() — пов'язати сокет з певною мережевою адресою;
- listen() — слухати повідомлення з мережі;
- accept() — коли клієнт підключився, сервер викликає метод accept(щоб прийняти або завершити підключення).

Клієнт викликає метод .connect(), щоб встановити з'єднання з сервером та ініціювати "тристороннє рукостискання" (процедура встановлення з'єднання). Крок рукостискання є важливим, оскільки він гарантує, що кожна сторона з’єднання доступна у мережі, іншими словами, що клієнт може зв’язатися з сервером та навпаки.

Посередині знаходиться розділ зворотного зв’язку, де відбувається обмін даними між клієнтом та сервером за допомогою викликів методів .send() та .recv().

Знизу клієнт та сервер закривають відповідні сокети.

#### Демонстраційний ехо-сервер та клієнт.

Для демонстрації роботи цього коду:
- відкрийте два термінали підряд — таким чином, щоб ви бачили їх одночасно;
- в першому з них запустіть код з файлу mono_server.py;
- в іншому терміналі запустіть код з файлу mono_client.py;
- поекспериментуйте з відправкою повідомлень від клієнта на сервер та отриманням ехо-повідомлень від серверу. Розберіться як працює код клієнта та сервера.

Цей клієнт та сервер можуть слугувати лише для демонстрації роботи технології й не мають суттєвої практичної цінності. Тому що сервер може обробляти лише одного клієнта та не має недоліки пов'язані з кількістю переданих даних (рядок "data_out = sock.recv(1024)", чому 1024?).

Цей код має два суттєвих питання:
- Як обробляти кілька підключень одночасно?
- Потрібно викликати методи .send() та .recv() поки всі дані не будуть відправлені або отримані (незалежно від їх кількості).

Це можна вирішити з використанням різних технологій розпаралелювання (thread/multiprocessing/asyncio). Це завдання виходить за рамки заняття, а у файлах multi_client.py та multi_server.py ви знайдете код, у якому реалізовані клієнт та сервер без цих недоліків, та цей код використовує модуль selectors зі стандартної бібліотеки Python, який дозволяє мультиплексувати події вводу\виводу досить високорівнево і незалежно від операційної системи.

### HTTP протокол

Ви вже побачили, що застосунки (два процеси), після встановлення з'єднання, повинні між собою багато про що домовлятися, щоб організувати обмін (наприклад, скільки байтів передавати, як фіксувати кінець спілкування, як одна сторона повідомить іншу, що щось пішло не так, або навпаки, що все йде за планом й таке інше). Якщо перейти у площину реальних завдань, то питань з'явиться значно більше: як передавати, які типи даних, як ідентифікувати свого "співрозмовника", як повідомляти про помилки, тощо.

Усі ці домовленості повинні відповідати якомусь протоколу, який буде формалізувати відносини між двома застосунками, які вступають у процедуру обміну інформацією. Наприклад, ви можете взяти за основу код для клієнта та для сервера, про які ми розмовляли у попередній частині (multi_client, multi_server) та придумати й додати функціональність, яка дасть можливість ідентифікувати кожного, фіксувати у мережі він зараз чи ні, обробляти якось відправку повідомлень один одному у різних ситуаціях (в мережі ваш кореспондент зараз чи ні), фіксувати доставку повідомлення та ще багато різних ситуацій — залежить від необхідності й фантазії. Це й називається — протокол обміну. Тобто, ви придумаєте й описуєте формальні правила обміну, формати даних, тощо. Це буде ваш спеціалізований протокол для досягнення якоїсь мети. Зараз існують сотні та тисячі протоколів для різноманітних завдань. Один з таких, надзвичайно популярних протоколів - [HTTP](https://uk.wikipedia.org/wiki/HTTP).

HTTP належить до протоколів моделі OSI 7-го прикладного рівня.

Основним призначенням протоколу HTTP — є передача веб-сторінок (текстових файлів з розміткою HTML), хоча за допомогою нього успішно передаються як інші файли, які пов'язані з веб-сторінками (зображення та застосунки), так і не пов'язані з ними (у цьому HTTP конкурує зі складнішим FTP).

HTTP припускає, що клієнтська програма — веб-браузер — здатна відображати гіпертекстові веб-сторінки та файли інших типів у зручній для користувача формі. Для правильного відображення HTTP дозволяє клієнтові дізнатися мову та кодування символів веб-сторінки й/або запитати версію сторінки у потрібних мові/кодуванні, використовуючи позначення зі стандарту MIME.


![http_intro](media/http_intro.png)

HTTP — протокол прикладного рівня, схожими на нього є FTP і SMTP. Обмін повідомленнями йде за звичайною схемою «запит-відповідь». Для ідентифікації ресурсів HTTP використовує глобальні URI. На відміну від багатьох інших протоколів, HTTP не зберігає свого стану. Це означає відсутність збереження проміжного стану між парами «запит-відповідь». Компоненти, що використовують HTTP, можуть самостійно здійснювати збереження інформації про стан, пов'язаний з останніми запитами та відповідями. Браузер, який посилає запити, може відстежувати затримки відповідей.

Кожен запит/відповідь складається з трьох частин:

- стартовий рядок;
- заголовки;
- тіло повідомлення, що містить дані запиту, запитаний ресурс або опис проблеми, якщо запит не виконано.


Обов'язковим мінімумом запиту є стартовий рядок. Починаючи з HTTP/1.1 обов'язковим став заголовок Host: (щоб розрізнити кілька доменів, які мають одну й ту ж IP-адресу).

![HTTP structure](media/HTTP_strucrure.png)

Якщо для того, щоб розібрати протокол детальніше, нам необхідно з'ясувати призначення декількох сутностей:

- метод HTTP;
- заголовки;
- статус відповіді.


#### Методи HTTP

HTTP визначає безліч методів запиту, які вказують, яка бажана дія виконається для цього ресурсу. Незважаючи на те, що їх назви можуть бути іменниками, ці методи запиту іноді називаються дієсловами HTTP.

Якщо дуже просто, то метод це те, що означає дію, яка повинна бути виконана сервером.

Наприклад, перейшовши за посиланням, яке вам переслав знайомий\знайома на якусь новину на сайті новин, то ваш браузер (клієнт) для переходу за цим посиланням надсилає до сайту новин (сервер) запис з HTTP методом GET.

Цей метод для сервера означає, що ви хочете отримати певну (це визначає URI) інформацію. Тобто кажете серверу — "надай мені цю інформацію". При цьому ви нічого не змінюєте на сервері — ви просто отримуєте щось (отримуєте доступ до існуючого ресурсу).

Якщо ви, прочитавши цю новину, хочете прокоментувати її (пропустимо зараз тему — ваших прав це робити та як сервер це визначає, про це трохи пізніше) — ви натискаєте якусь кнопку (щось на кшталт "додати коментар"), відкривається форма, у якій ви пишете свій коментар та коли він готовий, ви натискаєте кнопку "надіслати"\"надрукувати"\"створити" або щось подібне.

В цей час ваш браузер відправляє запит на сервер використовуючи метод POST. Цей метод наказує серверу створити новий ресурс (пам'ятаєте - коли ми переходили за посиланням на існуючу статтю — був метод GET та ми отримували доступ до існуючого ресурсу). Якщо ви маєте на це право, то сервер обробить ваш запит та створить новий ресурс — ваш коментар, до якого за посиланням на нього, використовуючи метод GET зможуть отримати доступ інші користувачі (якщо вони мають на це право — про це трохи пізніше).

Далі — ви читаєте свій коментар та бачите там помилку, яку хочете виправити. Метод GET дає можливість отримати доступ до ресурсу, метод POST — створити новий. Існують й методи PUT та PATCH, які наказують серверу оновити існуючий ресурс (треба самостійно розібратися яка між ними різниця).
Метод DELETE — наказує серверу видалити ресурс.

Це не всі методи, більше того, вони можуть створюватися, протокол не обмежує їх кількість. Але в основному використовуються саме методи перелічені вище.

Докладніше — [тут.](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods)


#### HTTP заголовки

Заголовки HTTP дозволяють клієнту та серверу надсилати додаткову інформацію з HTTP запитом або відповіддю. HTTP-заголовок містить не чутливу до регістру назву, а потім після (:) безпосередньо значення. Пробіли перед значенням ігноруються.

Простіше за все сприймати заголовки як набір метаданих (тобто даних, які описують основні дані :-) ) у форматі "ключ: значення". Наприклад, для запиту HTTP версії 1.1 обов'язковим став заголовок Host: який має значення імені хосту. Є стандартні заголовки, які використовує протокол HTTP для організації обміну, опису даних. Наприклад — заголовок Authorization містить облікові дані користувача, за якими сервер за допомогою системи авторизації може визначити хто цей користувач, та які має права (чи може створювати контент, редагувати його, мати доступ тощо).
Ви можете створювати свої заголовки, якщо у вас є така потреба.
Докладніше про це [тут.](https://developer.mozilla.org/ru/docs/Web/HTTP/Headers)


#### Коди статусів

Стартовий рядок відповіді містить у собі код статусу відповіді. Це цифровий код з трьох цифр, який просто визначає у відповіді сервера, яким чином він оцінює виконання вашого запиту. Наприклад, ви перейшли за посиланням, яке вам переслав ваш товариш на якусь новину (нагадую, що це означає, що ваш клієнт (браузер) надіслав на сервер (сайт новин) запит GET на отримання ресурсу (тієї статті якою поділився ваш товариш)). Сервер отримав ваш запит, обробивши його надіслав вам відповідь.

Якщо сервер вважає що все добре (він зрозумів ваш запит, перевірив ваші права на доступ до цього ресурсу, пошукав ресурс (за його URI) та знайшов його, а після цього надіслав вам його у відповідь у тілі відповіді), то ви отримаєте статус код, наприклад 200 — що означає — "Успішно". Запит успішно опрацьовано.

Якщо сервер визначить, що ви не маєте прав на доступ цієї інформації — ви отримаєте код 403 ("Заборонено". У клієнта немає прав доступу до вмісту, тому сервер відмовляється дати відповідь), якщо він не розуміє хто ви та не може розібратися маєте ви права на доступ до цієї інформації чи ні — ви отримаєте код 401 ("Неавторизовано". Для отримання запитаної відповіді потрібна автентифікація).

Якщо він не знайде за цим посиланням нічого (наприклад, автор або редакція видалила цю статтю) ви отримаєте, я гадаю, знайомий вам код, 404 ("Не знайдений". Сервер не може знайти запитуваний ресурс).


Статус-коди відповідей згруповані:
- інформаційні 100 – 199;
- успішні 200 – 299;
- перенаправлення 300 - 399;
- клієнтські помилки 400 – 499;
- серверні помилки 500 - 599.

Докладніше про статус-коди можна прочитати [тут](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

## Бібліотека requests та робота з запитами

Бібліотека requests є стандартом де-факто для створення HTTP-запитів у Python. Вона абстрагує складність створення запитів за красивим простим API, щоб ви могли зосередитися на взаємодії зі службами та споживанні даних у своїй програмі.

Бібліотека requests — не є вбудованою, тому потребує встановлення перед використанням.

In [ ]:
!pip install requests

In [3]:
import requests

Одним з найпоширеніших методів HTTP є GET. Метод GET вказує на те, що ви намагаєтеся отримати дані з вказаного ресурсу. Щоб зробити GET запит, викличте requests.get().
Наприклад:

In [4]:
requests.get('https://api.github.com')

<Response [200]>

Ми щойно зробили HTTP запит і отримали відповідь. Тепер давайте глибше все це розберемо. Збережемо відповідь у змінній:

In [5]:
response = requests.get('https://api.github.com')

Змінна response — це об'єкт типу requests.models.Response, який у своїх атрибутах зберігає багато корисної інформації, яка може бути нам використана. Наприклад, отримати код статусу можемо досить просто — звернувшись до атрибута .status_code

In [7]:
response.status_code

200

Іноді ви можете використовувати цю інформацію для прийняття рішень у своєму коді:

In [8]:
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

Success!


Бібліотека requests робить ще один крок далі, спрощуючи цей процес для вас. Якщо ви використовуєте Response екземпляр в умовному виразі, він оцінюватиме значення, True якщо код статусу був між 200 та 400 (тобто, умовно кажучи, відповіді про успішну обробку запиту — від 200 до 399, 400 не включається у відрізок) та False в іншому випадку.

Тому ви можете спростити останній приклад, переписавши if твердження:

In [ ]:
if response:
    print('Success!')
else:
    print('An error has occurred.')

Майте на увазі, що цей метод не перевіряє, що код стану дорівнює 200. Причина цього полягає в тому, що інші коди стану в межах діапазону 200 to 400, такі як 204 NO CONTENT та 304 NOT MODIFIED, також вважаються успішними в тому сенсі, що вони забезпечують деяку працездатну відповідь.

Наприклад, 204 повідомляє, що відповідь була успішною, але у тілі повідомлення немає вмісту для повернення.

Отже, переконайтеся, що ви використовуєте цей зручний скорочений запис, лише якщо ви хочете знати, чи запит був загалом успішним, а потім, якщо необхідно, обробіть відповідь належним чином на основі коду стану.

Припустимо, ви не хочете перевіряти код статусу відповіді в if виразі. Натомість ви хочете викликати виняток, якщо запит був невдалим. Ви можете зробити це за допомогою .raise_for_status():

In [9]:
import requests
from requests.exceptions import HTTPError

for url in ['https://api.github.com', 'https://api.github.com/invalid']:
    try:
        response = requests.get(url)

        # If the response was successful, no Exception will be raised
        response.raise_for_status()
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')  # Python 3.6
    except Exception as err:
        print(f'Other error occurred: {err}')  # Python 3.6
    else:
        print('Success!')

Success!
HTTP error occurred: 404 Client Error: Not Found for url: https://api.github.com/invalid


#### Тіло відповіді

Відповідь на GET запит часто містить певну цінну інформацію, відому як корисне навантаження, у тілі повідомлення. Використовуючи атрибути та методи Response, ви можете переглядати корисне навантаження в різних форматах.

Щоб переглянути вміст відповіді в bytes, ви використовуєте .content:

In [10]:
response = requests.get('https://api.github.com')
response.content

b'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sea

Хоча атрибут .content надає вам доступ до необроблених байтів корисного навантаження відповіді, ви часто захочете перетворити їх у рядок за допомогою кодування символів, наприклад UTF-8. Об'єкт response зробить це за вас, коли ви отримаєте доступ до тексту за допомогою атрибуту .text:

In [11]:
response.text

'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sear

Оскільки для декодування у bytes у str потрібна схема кодування, requests спробує вгадати кодування на основі заголовків відповіді (вони можуть бути метаданими які описують наші дані. Наприклад, заголовок Content-Type може описувати саме тип контенту і кодування Content-Type: text/html; charset=utf-8), якщо ви її не вкажете. Ви можете надати явне кодування, встановивши атрибут .encoding перед доступом до атрибуту .text:

In [12]:
response.encoding = 'utf-8' # Optional: requests infers this internally
response.text

'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sear

Якщо ви подивитеся на відповідь, ви побачите, що це насправді серіалізований вміст JSON. Щоб отримати словник, ви можете взяти str отриманий з .text і десеріалізувати його за допомогою json.loads(). Однак простішим способом виконання цього завдання є використання .json():

In [13]:
response.json()

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

Значення type, що повертається, .json() є словником, тому ви можете отримати доступ до значень в об’єкті за допомогою ключа.

З кодами стану та текстом повідомлень можна багато чого зробити. Але якщо вам потрібна додаткова інформація, наприклад, метадані про саму відповідь, вам потрібно буде переглянути заголовки відповіді.


#### Робота з заголовками

Заголовки відповіді можуть надати вам корисну інформацію, таку як тип вмісту корисного навантаження відповіді та ліміт часу для кешування відповіді. Щоб переглянути ці заголовки, викличте атрибут .headers:

In [14]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Sun, 09 Oct 2022 14:51:57 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src

Атрибут .headers повертає об’єкт, схожий на словник, що дозволяє отримати доступ до значень заголовка за ключем. Наприклад, щоб переглянути тип вмісту корисного навантаження відповіді, ви можете отримати доступ до Content-Type:

In [15]:
response.headers['Content-Type']

'application/json; charset=utf-8'

Однак у цьому об’єкті заголовків, схожому на словник, є щось особливе — він СХОЖИЙ НА СЛОВНИК, але не є словником у класичному розумінні python.

Специфікація HTTP визначає заголовки як нечутливі до регістру, що означає, що ми можемо отримати доступ до цих заголовків, не турбуючись про їх використання великими літерами:

In [16]:
response.headers['content-type']

'application/json; charset=utf-8'

#### Параметри рядка запиту

Одним із поширених способів налаштування GET запиту є передача значень через параметри рядка запиту в URL-адресі до серверу. Щоб зробити це за допомогою get(), ви передаєте дані в params. Наприклад, ви можете використовувати пошуковий API GitHub для пошуку requests бібліотеки:

In [17]:
# Search GitHub's repositories for requests
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
)

# Inspect some attributes of the `requests` repository
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')
print(f'Repository description: {repository["description"]}')

Repository name: grequests
Repository description: Requests + Gevent = <3


Що ми тільки що зробили?

Знаючи API GitHub (тобто вивчивши документацію до нього), ми сформували та передали на сервер GitHub запит з параметрами для пошуку відповідної інформації — репозиторію бібліотеки requests для мови python. Зараз для нас не важливо розуміння роботи API, для нас важливо як передати в запиті параметри. Ми зробили це за допомогою словника params.

Ви можете передати params до get() у вигляді словника, як ми щойно зробили, або у вигляді списку кортежів:

In [19]:
requests.get(
    'https://api.github.com/search/repositories',
    params=[('q', 'requests+language:python')],
)


<Response [200]>

Ви навіть можете передати значення як bytes:

In [ ]:
requests.get(
    'https://api.github.com/search/repositories',
    params=b'q=requests+language:python',
)

Рядки запиту корисні для параметризації GET запитів. Ви також можете налаштувати свої запити, додавши або змінивши заголовки, які ви надсилаєте.


#### Заголовки запитів

Щоб налаштувати заголовки, ви передаєте словник заголовків HTTP за get() допомогою headers параметра. Наприклад, ви можете змінити свій попередній пошуковий запит, щоб виділити відповідні пошукові терміни в результатах, вказавши text-match тип медіа в Accept заголовку:

In [21]:
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'},
)

# View the new `text-matches` array which provides information
# about your search term within the results
json_response = response.json()
repository = json_response['items'][0]
print(f'Text matches: {repository["text_matches"]}')

Text matches: [{'object_url': 'https://api.github.com/repositories/4290214', 'object_type': 'Repository', 'property': 'description', 'fragment': 'Requests + Gevent = <3', 'matches': [{'text': 'Requests', 'indices': [0, 8]}]}]


Заголовок Accept повідомляє серверу, які типи вмісту може обробляти ваша програма. У цьому випадку, оскільки ви очікуєте, що відповідні терміни пошуку будуть виділені, ви використовуєте значення заголовка application/vnd.github.v3.text-match+json, який є власним Accept заголовком GitHub, де вміст є спеціальним форматом JSON.

Перш ніж ви дізнаєтесь більше про способи налаштування запитів, давайте розширимо горизонт, вивчивши інші методи HTTP.


#### Інші методи HTTP

Окрім GET, інші популярні методи HTTP включають POST, PUT, DELETE, HEAD, PATCH та OPTIONS.

requests надає метод із сигнатурою, подібною до get(), для кожного з цих методів HTTP:

(httpbin.org — чудовий ресурс. Це служба, яка приймає тестові запити та надає у відповідь дані про запити. Наприклад, ви можете використовувати його для перевірки базових запитів:)


In [ ]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

Кожен виклик функції робить запит до httpbin служби за допомогою відповідного методу HTTP. Для кожного методу ви можете перевірити їх відповіді так само, як і раніше:

In [22]:
response = requests.head('https://httpbin.org/get')
response.headers['Content-Type']

'application/json'

In [23]:
response = requests.delete('https://httpbin.org/delete')
json_response = response.json()
json_response['args']

{}

#### Тіло повідомлення
Відповідно до специфікації HTTP запити POST, PUT та менш поширені PATCH передають свої дані через тіло повідомлення, а не через параметри у рядку запиту (як, наприклад, GET). Використовуючи requests, ви передасте корисне навантаження у відповідний data параметр функції.

data приймає словник, список кортежів, байтів або файлоподібний об’єкт. Ви захочете адаптувати дані, які ви надсилаєте в тексті свого запиту, до конкретних потреб служби, з якою ви взаємодієте.

Наприклад, якщо тип вмісту вашого запиту — application/x-www-form-urlencoded, ви можете надіслати дані форми у вигляді словника:

In [ ]:
requests.post('https://httpbin.org/post', data={'key':'value'})

Ви також можете надіслати ті самі дані як список кортежів:

In [ ]:
requests.post('https://httpbin.org/post', data=[('key', 'value')])

Однак, якщо вам потрібно надіслати дані JSON, ви можете скористатися json параметром. Коли ви передаєте дані JSON через json, requests ваші дані буде серіалізовано та додано правильний Content-Type заголовок.


In [24]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
json_response = response.json()
json_response['data']

'{"key": "value"}'

In [25]:
json_response['headers']['Content-Type']

'application/json'

Ви можете бачити з відповіді, що сервер отримав дані вашого запиту та заголовки, коли ви їх надсилали. requests також надає вам цю інформацію у формі PreparedRequest.


Коли ви робите запит, requests бібліотека готує запит перед фактичним надсиланням на сервер призначення. Підготовка запиту включає такі речі, як перевірка заголовків та серіалізація вмісту JSON.

Ви можете переглянути PreparedRequest (тобто, а що ми відправляли в запиті?):

In [26]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
response.request.headers['Content-Type']

'application/json'

In [27]:
response.request.url

'https://httpbin.org/post'

In [28]:
response.request.body

b'{"key": "value"}'

Перевірка PreparedRequestдає вам доступ до всіх видів інформації про зроблений запит, наприклад корисного навантаження, URL-адреси, заголовків, автентифікації тощо.



#### Аутентіфікація

Автентифікація допомагає службі зрозуміти, хто ви. Зазвичай ви надаєте свої облікові дані серверу, передаючи дані через Authorization заголовок або спеціальний заголовок, визначений службою. Усі функції запиту, які ви бачили до цього моменту, надають параметр під назвою auth, який дозволяє вам передати свої облікові дані.

Одним із прикладів API, який вимагає автентифікації, є API автентифікованого користувача GitHub. Ця кінцева точка надає інформацію про профіль автентифікованого користувача. Щоб зробити запит до API автентифікованого користувача, ви можете передати своє ім’я користувача та пароль GitHub у кортежі до get():

__!!!зверніть увагу — зараз Вам буде необхідно ввести свої облікові дані на GitHub__

In [33]:
from getpass import getpass
requests.get('https://api.github.com/user', auth=('username', getpass()))

<Response [401]>

Запит виконано успішно, якщо облікові дані, які ви передали в кортежі auth, дійсні. Якщо ви спробуєте зробити цей запит без облікових даних, ви побачите, що код статусу 401 Unauthorized.


Коли ви передаєте своє ім’я користувача та пароль у кортежі до auth-параметра, requests облікові дані застосовуються за допомогою [основної схеми автентифікації](https://en.wikipedia.org/wiki/Basic_access_authentication) доступу HTTP під капотом. Важливо розуміти: це досить застаріла та примітивна схема, вона не забезпечує достатній рівень безпеки. На сьогодні існують інші схеми (наприклад - [OAuth2](https://oauth.net/2/)) — більш складні, але значно більш надійні. Але така схема автентіфікації існує та може використовуватись.

Таким чином, ви можете зробити такий самий запит, передавши явні облікові дані базової автентифікації за допомогою HTTPBasicAuth:

In [32]:
from requests.auth import HTTPBasicAuth
from getpass import getpass
requests.get(
    'https://api.github.com/user',
    auth=HTTPBasicAuth('username', getpass())
)

<Response [401]>

#### Тайм-аути

Коли ви робите вбудований запит до зовнішньої служби, вашій системі потрібно буде дочекатися відповіді, перш ніж рухатися далі. Якщо ваша програма надто довго чекає такої відповіді, запити до вашої служби можуть виконуватися резервно, ваші враження від користувача можуть погіршитися, або ваші фонові завдання можуть зависати.

За замовчуванням requests чекатиме відповіді необмежений час, тому майже завжди слід вказувати тривалість очікування, щоб запобігти цьому. Щоб встановити час очікування запиту, використовуйте timeout-параметр. timeout може бути цілочисельним або числом з плаваючою крапкою, що означає кількість секунд очікування відповіді до завершення тайм-ауту:

In [ ]:
requests.get('https://api.github.com', timeout=1)

requests.get('https://api.github.com', timeout=3.05)

У першому запиті запит закінчиться через 1 секунду. У другому запиті час очікування закінчиться через 3,05 секунди.

Ви також можете передати кортеж, у timeout якому перший елемент буде тайм-аутом з’єднання (час, протягом якого клієнт може встановити з’єднання із сервером), а другий — тайм-аут читання (час, протягом якого він чекатиме на відповідь після того, як клієнт встановив з'єднання):

In [ ]:
requests.get('https://api.github.com', timeout=(2, 5))

#### Об’єкт сеансу

Досі ви мали справу з requests API високого рівня, такими як get() і post(). Ці функції є абстракціями того, що відбувається, коли ви робите свої запити. Вони приховують деталі реалізації, наприклад, як керуються з’єднаннями, тому вам не потрібно про них турбуватися.

Під цими абстракціями знаходиться клас під назвою Session. Якщо вам потрібно точно налаштувати контроль над тим, як надсилаються запити, або покращити продуктивність ваших запитів, вам може знадобитися Session напряму використовувати примірник.

Сеанси використовуються для збереження параметрів у запитах. Наприклад, якщо ви хочете використовувати ту саму автентифікацію для кількох запитів, ви можете використовувати сеанс:

In [ ]:
import requests
from getpass import getpass

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())

Щоразу, коли ви робите запит за допомогою session, після ініціалізації облікових даних автентифікації облікові дані зберігатимуться.

Основна оптимізація продуктивності сеансів здійснюється у формі постійних з’єднань. Коли ваша програма під'єднується до сервера за допомогою Session, вона зберігає це з’єднання у пулі з’єднань. Коли ваша програма знову захоче під'єднатися до того самого сервера, вона повторно використовуватиме з’єднання з пулу, а не встановлюватиме нове.